# Задание 1

## Состояния
Состояния представлены в виде списка символов.

In [45]:
states = ['h','c']

## Переходы
Переходы представлены в виде словаря, где ключи — это кортежи, а значения — вероятности перехода. Первый элемент кортежа — это состояние, из которого мы переходим, а второй элемент — это состояние, в которое мы переходим.

In [46]:
transitions = {('h','h') : 0.6,
               ('h','c') : 0.4,
               ('c','h') : 0.5,
               ('c','c') : 0.5,
    }

## Матрицы B1, B2
Матрицы представлены в виде словаря, где ключи — это состояния погоды, а значения — словари. Внутренние словари содержат эмитированные символы в качестве ключей и вероятности 1, 2, 3 в качестве значений.

In [47]:
emissions = {'h' : {'1':0.2, '2':0.4, '3':0.4},
             'c' : {'1':0.5, '2':0.4, '3':0.1}
    }

## Последовательность состояний
Последовательность — это строка.

In [35]:
sequence = '1332333212113232213311121'

## Инициализация матриц

In [5]:
def initialize_matrix(dim1, dim2, value=0):
    F = []
    for i in range(0, dim1):
        F.append([])
        for j in range(0, dim2):
            F[i].append(value)
    return F

In [111]:
def print_matrix(matrix, axis1, axis2):
    w = '{:<10}'
    print(w.format('') + ''.join([w.format(char) for char in axis2]))
    for i, row in enumerate(matrix):
        print(w.format(axis1[i]) + ''.join(['{:<10.2e}'.format(item) for item in row]))

In [193]:
F = initialize_matrix(len(states), len('313'))

print_matrix(F, states, '313')

          3         1         3         
h         0.00e+00  0.00e+00  0.00e+00  
c         0.00e+00  0.00e+00  0.00e+00  


In [132]:
from typing import List, Dict, Tuple

def forward(states: List[str], 
            transitions: Dict[Tuple[str, str], float], 
            emissions: Dict[str, Dict[str, float]],
            sequence: str,
            pi: List[float],
           ) -> float:
    
    F = initialize_matrix(len(states), len(sequence))
    for i in range(len(states)):
        F[i][0] = pi[i] * emissions[states[i]][sequence[0]]
    for j in range(1, len(sequence)): # бегу по последовательности
        for i in range(len(states)): # бегу по статусам
            p_sum = 0
            for k in range(len(states)): # просматриваю все предыдущие статусы
                p_sum += F[k][j-1]*transitions[(states[k], states[i])] * emissions[states[i]][sequence[j]]
            F[i][j] = p_sum
    p_sum = 0
    for k in range(0, len(states)):
        p_sum += F[k][len(sequence) - 1]
    return p_sum

In [133]:
forward(states, transitions, emissions, sequence, pi=[.8, .2])

4.978300204739011e-13

# Задание 2

In [189]:
from typing import List, Dict, Tuple

def viterbi(states: List[str], 
            transitions: Dict[Tuple[str, str], float], 
            emissions: Dict[str, Dict[str, float]],
            sequence: str,
            pi: List[float],
           ) -> Tuple[float, List[str]]:

    F = initialize_matrix(len(states), len(sequence)) #заводим матрицу для хранения вероятностей
    FP = [] #здесь хранится макс. вероятности для каждого элемента sequence в виде кортежей, колиечество которых = len(states)
    
    for i in range(len(states)): #обрабатываем переход start -> 1
        F[i][0] = pi[i] * emissions[states[i]][sequence[0]]
    values = []
    for i in range(len(states)):
                values.append((F[i][0], i))
    FP.append(max(values, key=lambda x: x[0]))
    
    for j in range(1, len(sequence)):
        values_sequence = []
        for i in range(len(states)):
            values_states = []
            for k in range(len(states)):
                #перебираем все вероятности попадания в состояние i, j из разных состояний states
                values_states.append(F[k][j-1]*transitions[(states[k], states[i])] * emissions[states[i]][sequence[j]])
            max_val = max(values_states) 
            F[i][j] = max_val #выбираю самую макс. вероятность попадания в i, j из разных состояний states
            values_sequence.append((max_val, i))
        FP.append(max(values_sequence, key=lambda x: x[0]))

    probable_sequence = []
    for i in range(len(sequence)):
        probable_sequence.append(states[FP[i][1]])
    
    return FP[-1][0], probable_sequence

In [190]:
viterbi(states, transitions, emissions, sequence, pi=[.8, .2])

(7.30406948721132e-17,
 ['h',
  'h',
  'h',
  'h',
  'h',
  'h',
  'h',
  'h',
  'c',
  'h',
  'c',
  'c',
  'h',
  'h',
  'h',
  'h',
  'h',
  'c',
  'h',
  'h',
  'c',
  'c',
  'c',
  'h',
  'c'])